Exploratory Analysis
====================

First try at pulling the game data down and exploring how the website is organized.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy
import pandas
import seaborn

from bs4 import BeautifulSoup
import requests

In [2]:
test_site = 'http://results.worldcurling.org/Championship/Details/555'

In [3]:
r = requests.get(test_site)
assert r.status_code == 200

soup = BeautifulSoup(r.text, 'html.parser')

Game Scores
-----------

From my quick search before setting up this notebook, I was only pulling the final scores, not the box scores... Let's see if I can figure that out.

In [5]:
scores = soup.find_all('td', class_='col-md-2')
print(len(scores))
print(scores[0])

88
<td class="col-md-2 text-center">
(1)                            <br/>
SWE - JPN                            <br/>
8 - 5                    </td>


In [12]:
data = scores[0].text.split()
game_data = {'draw': int(data[0].lstrip('(').rstrip(')')),
             'team1': data[1], 'team2': data[3],
             'team1_score': int(data[4]), 'team2_data': int(data[6])}
print(game_data)

{'team2': 'JPN', 'team1': 'SWE', 'team2_data': 5, 'team1_score': 8, 'draw': 1}


In [24]:
def create_game_data(game_soup):
    data = game_soup.text.split()
    if len(data) > 1:
        draw = data[0].lstrip('(').rstrip(')')
        game_data = {'team1': data[1], 'team2': data[3],
                     'team1_score': int(data[4]), 'team2_data': int(data[6])}
        try:
            game_data['playoffs'] = False
            draw = int(draw)
            game_data['draw'] = 'D{}'.format(draw)
        except:
            game_data['playoffs'] = True
            game_data['draw'] = draw
        return game_data

tourney_data = [create_game_data(s) for s in scores]
tourney_data = [d for d in tourney_data if d is not None]
print(len(tourney_data))  # to see how many false positives we got from our soup.find_all

71
